# Scraping "terre d'avenir"

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
from serde import serialize, deserialize
from serde.json import to_json, from_json
import pandas as pd
from requests import get 
from dataclasses import dataclass


In [7]:
nav = webdriver.Chrome()

In [8]:
adresse = "https://www.terdav.com/ps-france/tp-voyage-famille#nos-voyages"
nav.get(adresse)
# code = nav.page_source

In [9]:
bouton = nav.find_element(By.XPATH, '//*[@id="axeptio_btn_acceptAll"]')
bouton.screenshot("bouton1.png")

True

In [10]:
bouton.click()

## catégorie en famille 

In [12]:
bouton2 = nav.find_element(By.XPATH, '//*[@id="blockResultats"]/a')
bouton2.screenshot("bouton2.png")

True

In [16]:
bouton2.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x008A6903+2517251]
	Ordinal0 [0x0083F8E1+2095329]
	Ordinal0 [0x00742710+1058576]
	Ordinal0 [0x0076E324+1237796]
	Ordinal0 [0x00764037+1196087]
	Ordinal0 [0x007864D3+1336531]
	Ordinal0 [0x00763A36+1194550]
	Ordinal0 [0x007865BA+1336762]
	Ordinal0 [0x00795BBF+1399743]
	Ordinal0 [0x0078639B+1336219]
	Ordinal0 [0x007627A7+1189799]
	Ordinal0 [0x00763609+1193481]
	GetHandleVerifier [0x00A35904+1577972]
	GetHandleVerifier [0x00AE0B97+2279047]
	GetHandleVerifier [0x00936D09+534521]
	GetHandleVerifier [0x00935DB9+530601]
	Ordinal0 [0x00844FF9+2117625]
	Ordinal0 [0x008498A8+2136232]
	Ordinal0 [0x008499E2+2136546]
	Ordinal0 [0x00853541+2176321]
	BaseThreadInitThunk [0x75D76739+25]
	RtlGetFullPathName_UEx [0x779B8AFF+1215]
	RtlGetFullPathName_UEx [0x779B8ACD+1165]


In [17]:
code = nav.page_source

In [18]:
soupe = BS(code, "lxml")
cells = soupe.find_all(name="article", attrs={"class":"circuit item"})

In [19]:
len(cells)

65

In [45]:
motif = re.compile("name.*?:(.*?),|price.*?:(.*?),|dimension6.*?:(.*?),|dimension7.*?:(.*?),|dimension8.*?:(.*?),|dimension12.*?:(.*)")
feat_val = list()

for i in range(0,len(cells)):
    script = cells[i].script
    texte = script.text
    indice = texte.find("data")
    data = texte[indice:]
    #data = data.replace(" ","")
    index = 0
    for j in motif.finditer(data):
        c = j.groups()
        feat_val.append(c[index])
        index = index +1

In [46]:
# code opti trouvé sur le net

feat_val2 = list( zip(*[iter(feat_val)]*6))

In [47]:
feat_val2

[('   "plaisirs neige &#224; sixt-fer-&#224;-cheval"',
  '   690',
  '   "alpes du nord"',
  "   '8 Jours'",
  "   'niveau 2'",
  '   "multi-activités"'),
 ('   "haute clar&#233;e &quot;espace trappeur&quot;"',
  '   860',
  '   "alpes du sud"',
  "   '7 Jours'",
  "   'niveau 1'",
  '   "multi-activités"'),
 ('   "aventures hivernales sur le plateau de l&#39;aubrac"',
  '   720',
  '   "massif central"',
  "   '6 Jours'",
  "   'niveau 1'",
  '   "multi-activités"'),
 ('   "neige et soleil du queyras en famille"',
  '   730',
  '   "alpes du sud"',
  "   '7 Jours'",
  "   'niveau 1'",
  '   "multi-activités"'),
 ('   "plaisirs neige dans les aravis"',
  '   830',
  '   "alpes du nord"',
  "   '7 Jours'",
  "   'niveau 1'",
  '   "multi-activités"'),
 ('   "vall&#233;e de la roya',
  '   670',
  '   "alpes du sud"',
  "   '7 Jours'",
  "   'niveau 2'",
  '   "canyoning"'),
 ('   "davy crockett en haute bl&#233;one"',
  '   730',
  '   "alpes du sud"',
  "   '6 Jours'",
  "   'niveau 1'

In [3]:
@serialize
@deserialize
@dataclass
class Sejour :
    description : str
    prix : int
    lieux : str
    duree : int
    diff : str
    theme : str 

In [89]:
resultats = list()
for i in feat_val2:
    description = i[0]
    description = description[4:-1]
    prix = int(i[1])
    lieu = i[2]
    lieu = lieu[4:-1]
    a, b, c, d, duree, *_ = i[3]
    duree = int(duree)
    diff = i[4]
    diff = diff[4:-1] 
    theme = i[5]
    theme = theme[4:-1]
    
    resultats.append(
        Sejour(
            description,
            prix,
            lieu,
            duree,
            diff,
            theme
        )
    )

In [90]:
resultats

[Sejour(description='plaisirs neige &#224; sixt-fer-&#224;-cheval', prix=690, lieux='alpes du nord', duree=8, diff='niveau 2', theme='multi-activités'),
 Sejour(description='haute clar&#233;e &quot;espace trappeur&quot;', prix=860, lieux='alpes du sud', duree=7, diff='niveau 1', theme='multi-activités'),
 Sejour(description='aventures hivernales sur le plateau de l&#39;aubrac', prix=720, lieux='massif central', duree=6, diff='niveau 1', theme='multi-activités'),
 Sejour(description='neige et soleil du queyras en famille', prix=730, lieux='alpes du sud', duree=7, diff='niveau 1', theme='multi-activités'),
 Sejour(description='plaisirs neige dans les aravis', prix=830, lieux='alpes du nord', duree=7, diff='niveau 1', theme='multi-activités'),
 Sejour(description='vall&#233;e de la roy', prix=670, lieux='alpes du sud', duree=7, diff='niveau 2', theme='canyoning'),
 Sejour(description='davy crockett en haute bl&#233;one', prix=730, lieux='alpes du sud', duree=6, diff='niveau 1', theme='mul

In [91]:
with open("backup_terdav.json", "w") as fichier:
    fichier.write(to_json(resultats))

## catégorie liberté

In [94]:
bouton = nav.find_element(By.XPATH, '//*[@id="blockFiltre"]/div[2]/label[3]')
bouton.screenshot("bouton1.png")

True

In [95]:
bouton.click()

In [98]:
bouton2 = nav.find_element(By.XPATH, '//*[@id="blockResultats"]/a')
bouton2.screenshot("bouton2.png")

True

In [105]:
bouton2.click()

In [106]:
code = nav.page_source

In [107]:
soupe = BS(code, "lxml")
cells = soupe.find_all(name="article", attrs={"class":"circuit item"})

In [108]:
len(cells)

136

In [109]:
motif = re.compile("name.*?:(.*?),|price.*?:(.*?),|dimension6.*?:(.*?),|dimension7.*?:(.*?),|dimension8.*?:(.*?),|dimension12.*?:(.*)")
feat_val = list()

for i in range(0,len(cells)):
    script = cells[i].script
    texte = script.text
    indice = texte.find("data")
    data = texte[indice:]
    #data = data.replace(" ","")
    index = 0
    for j in motif.finditer(data):
        c = j.groups()
        feat_val.append(c[index])
        index = index +1

In [110]:
feat_val2 = list( zip(*[iter(feat_val)]*6))

In [111]:
resultats = list()
for i in feat_val2:
    description = i[0]
    description = description[4:-1]
    prix = int(i[1])
    lieu = i[2]
    lieu = lieu[4:-1]
    a, b, c, d, duree, *_ = i[3]
    duree = int(duree)
    diff = i[4]
    diff = diff[4:-1] 
    theme = i[5]
    theme = theme[4:-1]
    
    resultats.append(
        Sejour(
            description,
            prix,
            lieu,
            duree,
            diff,
            theme
        )
    )

In [112]:
resultats

[Sejour(description='de rennes &#224; saint-malo &#224; v&#233;lo', prix=270, lieux='bretagne - normandie', duree=3, diff='niveau 1', theme='vélo'),
 Sejour(description='a v&#233;lo le long des plages du d&#233;barquement', prix=290, lieux='bretagne - normandie', duree=4, diff='niveau 1', theme='vélo'),
 Sejour(description='escapade royale au pays des ch&#226;teaux en famille', prix=325, lieux='vallée de la loire', duree=4, diff='niveau 1', theme='vélo'),
 Sejour(description='les calanques de marseille en libert&#233;', prix=330, lieux="provence - côte d'azur", duree=4, diff='niveau 2', theme='randonnée'),
 Sejour(description='balades avec un &#226;ne au pied du vercors', prix=340, lieux='alpes du nord', duree=4, diff='niveau 2', theme='randonnée avec âne'),
 Sejour(description='la provence &#224; v&#233;l', prix=350, lieux="provence - côte d'azur", duree=4, diff='niveau 1', theme='vélo'),
 Sejour(description='escapade v&#233;lo au cœur des grands ch&#226;teaux', prix=365, lieux='vallé

In [113]:
with open("backup_terdav2.json", "w") as fichier:
    fichier.write(to_json(resultats))